# Binäre Klassifikation anhand eines Datensatzes zur Hautfarbenerkennung

## Thema
In dieser Binären Klassifikation handelt es sich um die Vorhersage auf das Eintreffen einer Hautfarbe eines Menschen aus der Kombination der Primärfarben. Um das Herauszufinden wird ein Datensatz (der mehrere Farbkombination) enthält verwendet.

## Technische Vorbereitung
Damit alle notwendigen Funktionen und Befehle zur Verfügung stehen, müssen zunächst einige Module mit ihren jeweiligen Klassen importiert werden. Dazu zählt pandas und scikit-learn. Alle Module werden für spätere Datenanalysemethoden genutzt.


In [36]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

## Analyse des Datensatzes
Der Datensatz liegt hierbei in einer ".csv" - Datei und beinhaltet 2500 Datensätze. Er ist in drei Spalten untergliedert. Die Spalte 0 entspricht der Farbe blau, Spalte 1 entspricht der Farbe grün, Spalte 2 entspricht der Farbe rot und die Spalte 3 beinhaltet die beiden Gruppen in Form von Ziffern. Dabei steht die 1 für Hautfarbe und die 2 definiert eine Farbkombination, die keine Hautfarbe ergibt.


In [44]:
dataset = pd.read_csv("Skin_NonSkin.csv", header=None)
print(dataset.head())
print(dataset.shape)
print("")
print(dataset.describe())


     0    1    2  3
0  176  174  126  2
1  153  191  255  2
2   57   59   23  2
3  198  198  158  2
4  178  175  131  2
(2500, 4)

                 0            1            2            3
count  2500.000000  2500.000000  2500.000000  2500.000000
mean    124.656400   132.313600   125.663200     1.776000
std      61.985437    60.021465    71.977355     0.417005
min       0.000000     0.000000     0.000000     1.000000
25%      70.000000    87.000000    82.000000     2.000000
50%     138.000000   153.000000   129.000000     2.000000
75%     175.000000   177.000000   168.000000     2.000000
max     255.000000   255.000000   255.000000     2.000000


Sowohl die Farben- als auch die Gruppen-Spalte besitzen den Datentyp int64.

In [38]:
print(dataset.dtypes)


0    int64
1    int64
2    int64
3    int64
dtype: object


Durch den folgenden Code wurde die Anzahl von Hautfarben bzw. keiner Hautfarbe in dem Datensatz gezählt. Dabei fällt auf, dass es sehr unausgeglichen ist und es sich bei den meisten Kombinationen um keine Hautfarbe handelt.

In [45]:
print(dataset.groupby(3).size())



3
1     560
2    1940
dtype: int64


## Trainings- und Testdaten
Im nächsten Schritt werden die Daten in den Spalten nach Primärfarben und Hautfarbe gruppiert. Die Trainingsdaten werden für das Lernen der Muster und Zusammenhänge in den Daten verwendet. Der Algorithmus nutzt diese Daten, um daraus zu lernen. Bei den Testdaten handelt es sich um Daten mit der gleichen Wahrscheinlichkeitsverteilung. Diese Daten werden vom Algorithmus vorher nicht genutzt. Mit ihnen kann nachgeweisen werden, mit welcher Qualität der Algorithmus auf neue Daten reagieren wird. Die Verteilung splittet sich auf 80% Trainings- und 20% Testdaten.

In [46]:
x = dataset.values[:,0:3].astype(float)
y = dataset.values[:,3]

x_train,y_train = x[:2000],y[:2000]
x_test,y_test = x[2000:],y[2000:]

print(x_train.shape)
print(x_test.shape)


(2000, 3)
(500, 3)


## Evaluierung der Algorithmen
Im folgenden Code wird mit Hilfe der Genauigkeit der beste Algorithmus herausgefunden. Die Genauigkeit wird dabei anhand des Mittelwertes und der Standardabweichung gemessen. Es stellt sich heraus, dass der Gauß-Algorithmus und der Support-Vector-Machine-Klassifikator am besten geeignet sind.

In [6]:
models = []
models.append(("LR", LogisticRegression()))
models.append(("NB", GaussianNB()))
models.append(("SVM", SVC()))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.915500 (0.018768)
NB: 0.922000 (0.019261)
SVM: 0.986500 (0.006344)


## Evaluierung der Algorithmen mit Hilfe der Standardisierung
In diesem Code werden alle Merkmale so transformiert, dass sie einen Mittelwert von 0 und eine Standardabweichung von 1 besitzen. Durch die Verwendung von pipelines wird Datenverlust vermieden. Dabei fällt auf, dass die Genauigkeit bei allen Algorithmen gestiegen ist. Der Gauß-Algorithmus und der Support-Vektor-Machine-Klassifikator haben weiterhin die größte Genauigkeit.

In [19]:
pipelines = []
pipelines.append(("ScaledLR", Pipeline([("Scaler", StandardScaler()), ("LR", LogisticRegression())])))
pipelines.append(("ScaledNB", Pipeline([("Scaler", StandardScaler()), ("NB", GaussianNB())])))
pipelines.append(("ScaledSVM", Pipeline([("Scaler", StandardScaler()), ("SVM", SVC())])))

results =[]
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


ScaledLR: 0.919250 (0.003902)
ScaledNB: 0.924500 (0.005213)
ScaledSVM: 0.994975 (0.001301)


## Verbesserung der Algorithmen
Durch das Verändern/Anpassen von Parametern für einen jeweiligen Algroithmus kann seine Genauigkeit erhöht werden.

### Verbesserung des Support-Vector-Machine-Klassifikator
Beim SVM lassen sich zwei Parameter anpassen. Zum einen der kernel und zum anderen der C-Wert. Der kernel versucht den Trainingsvektor in verschiedene höherdimensionale Räume zu überführen, damit die Vektoren sich leichter trennen lassen. Beim C-Wert wird dem Klassifikator mitgeteilt, wie sehr er eine Fehlklassifizierung vermeiden soll. Hierbei entsteht eine Genauigkeit von 99,75% mit dem C-Wert von 255 und dem Standard kernel rbf (Radial Basis Function).

In [11]:
scaler = StandardScaler().fit(x_train)
standardX = scaler.transform(x_train)
c_values = [0.1, 5, 25, 100, 255]
kernel_values = ["linear", "poly", "rbf", "sigmoid"]
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=10, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", cv=kfold)
grid_result = grid.fit(x_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.997500 using {'C': 255, 'kernel': 'rbf'}


## Random Forest
Hierbei handelt es sich um einen Meta-Schätzer. Er passt mehrere Entscheidungsbaumklassifikatoren für verschiedene Teilstichproben des Datensatzes an. Dazu bildet er einen Mittelwert um die Genauigkeit zu verbessern. Des Weiteren gehört Random Forest zu den Bagging Mehtoden. Sie kombinieren verschiedene Vorhersagen anderer Algorithemen um eine genauere Vorhersage zu treffen. Am Ende ergibt sich eine Genauigkeit von 99,3%.

In [10]:
model = RandomForestClassifier()
kfold = KFold(n_splits=10, shuffle=True)
cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring="accuracy")
msg = "RF: %f (%f)" % (cv_results.mean(), cv_results.std())
print(msg)  

RF: 0.993000 (0.005099)


## Vorhersage
Die beste Genauigkeit erzielte der Support-Vektor-Machine-Klassifikator mit den Parametern: 

    C-Wert: 255
    kernel: rbf
Im folgenden Code wird der Klassifikator mit den Testdaten überprüft, um eine Genauigkeit der Vorhersage neuer Werte zu treffen. Die Werte ähneln der vordefinierten Genauigkeit des Support-Vektor-Machine-Klassifikator, sie beträgt 100%.

In [47]:
model = SVC(C=255, kernel="rbf")
model.fit(standardX, y_train)
validateX = scaler.transform(x_test)
predictions = model.predict(validateX)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))



1.0
[[128   0]
 [  0 372]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       128
           2       1.00      1.00      1.00       372

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



## Quellen
https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

https://machinelearningmastery.com/bagging-and-random-forest-ensemble-algorithms-for-machine-learning/

Brownlee, Jason (2018): Machine Learning Mastery with Python, S. 144 - 161 